# Exercise 7C - Parametric Analysis

In this exercise, it will be shown how to complete a parametric analysis for an EnergyPlus parameter. This is applicable to the parametric/sensitivity analysis portion of your coursework.

### Colour codes

<span style="color:orange;"> Orange text is for emphasis and definitions </span>

<span style="color:lime;"> Green text is for tasks to be completed by the student </span>

<span style="color:dodgerblue;"> Blue text is for Python coding tricks and references </span>

## Load all the necessary Python packages
All packages should work with Conda environment if installed on your machine. Otherwise all necessary packages can be installed in a virtual environment (.venv) in VS Code using: Ctrl+Shift+P > Python: Create Environment > Venv > Python 3.12.x > requirements.txt

In [ ]:
import itertools
import json
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
import os
from pathlib import Path
import pandas as pd
import time

from src.runEnergyPlus import run_energyPlus


## 1. Getting Started
### 1.1 Enter the general parameters for this run.

<span style="color:lime;"> These should be the same from previous exercises. Ensure they are correct.</span>


In [ ]:
# Enter a save name for this run
saveName = "Exercise_7C"

# Enter the path to the directory with your EnergyPlus executable. Enter the full path separated by commas.
ep_dir = Path("c:\\", "EnergyPlusV25-1-0")

# The weather file to be used for this batch of simulations. This file should be located in the src/weatherData/ directory.
weatherFile = "GBR_ENG_London.Wea.Ctr-St.James.Park.037700_TMYx.2004-2018.epw"

# The baseline file to be used for this simulation. This file should be located in the idfs/ directory
idf_file = "1-storey_baseline.idf"


Create the full paths for the idf and weather files and confirm they both exist. Else an exception will be created.

In [ ]:
baseline_idf_path = Path("idfs", idf_file)
weather_file_path = Path("weatherData", weatherFile)

if not ep_dir.exists():
    raise Exception (f"Could not find energyPlus executable at {ep_dir}.")
if not baseline_idf_path.exists():
    raise Exception (f"Could not find idf_file at {baseline_idf_path}.")
if not weather_file_path.exists():
    raise Exception (f"Could not find weather_file at {weather_file_path}.")

print (f"The EnergyPlus directory is: {ep_dir}.")
print (f"The baseline idf file is: {baseline_idf_path}.")
print (f"The weather file is: {weather_file_path}.")

## 1. Setting Up a Parametric Analysis

This step demonstrates how to run a parametric analysis beginning with one parameter (height). <span style = "color:lime;"> The code will be demonstrated for one parameter. Students will adapt the code to perform parametric analyses for the remaining parameters. </span>

For this exercise we will assume that the following parameters remain constant throughout this exercise and the coursework. Don't do a parametric analysis for these parameters:
* coolingSetpoint (set to 99 &deg;C)
* length
* width

### 1.1 Parametric Analysis Steps
1. Choose a parameter to analyse
2. Select an appropriate range of values for that parameter
3. Set all other variables to their *"default"* values
4. Perform Runs

***Step 1 is done! We will be studying the effect of building height.***

### 1.2 Select an Appropriate Range of Values
From the lecture:
* Minimum of three to assess curvature
* Preference for a minimum of five
* Range determined on reasonable judgment

The range (min and max value) need to be determined. Often there is not a right or wrong answer for what numbers to choose. It may require some engineering judgment, experience and research to determine reasonable values.

From my knowledge of buildings, new residential buildings typically have a minimum room height of 2.4 m (the size of an 8ft sheet of drywall). Other houses may have some shorter or taller ceiling heights.

<span style = "color:lime;"> Do you agree with the choices made here? </span>

In [ ]:
parameterName = "height"
n_simulations = 5

min_value = 2.4 # [m]
max_value = 4 # [m]

Typically, you will divide the range into equally spaced intervals. You know a practical method for doing that.

In [ ]:
values = np.linspace(min_value, max_value, n_simulations)
print (f"The values being used for the {parameterName} parameter are {values}.")

### 1.3 Set all other variables to their *"default"* values
From the lecture:
* Usually halfway between their min and max range
* Note this can influence conclusions

I have prepared a list of default variables for this exercise. They are located in *simulationParameters/Exercise 7C.json*. <span style = "color:lime;"> Do you agree with the choices made? </span>

In [ ]:
# The parameters file to be used as part of this simulation
parameters_file = "Exercise 7C.json"

parameters_file_path = Path("simulationParameters", parameters_file)

if not parameters_file_path.exists():
    raise Exception (f"Could not find the parameters_file at {parameters_file_path}.")

with open (parameters_file_path) as f:
    parameters = json.load(f)

print (f"Name                      TYPE        VALUES")
for k,v in parameters.items():
    print (f"{k:<26}{v['type']:<12}{v['values']}")

You can simply modify the parameters dictionary with the intended values for your parametric analysis.

In [ ]:
parameters[parameterName]["values"] = values

Confirming that the changes have been made.

In [ ]:
print (f"Name                      TYPE        VALUES")
for k,v in parameters.items():
    print (f"{k:<26}{v['type']:<12}{v['values']}")

### 1.4 Perform the simulations
We will run the simulations in parallel. Before doing that, we will prepare the combinations of parameters we want to run.

Here, we can recycle some of the full-factorial code. We will generate it using the cartesian product. Because there is only one value in most parameters, it will result in a short list of combinations

### 1.4.1 Create the Combinations

In [ ]:
# List comprehension to access the values for each parameters dictionary
v = [x["values"] for x in parameters.values()]
combinations = list(itertools.product(*v))

# Place the resulting combinations into a dataframe and save
combinations = pd.DataFrame(combinations, columns = parameters.keys())
print (combinations)

# Save the combinations as a csv
savePath = Path("outputs", "combinations", f"combinations_Exercise7C_{parameterName}.csv")
combinations.to_csv(savePath)
print (f"Combinations dataframe saved to {savePath}.")


### 1.4.2 Run EnergyPlus
The first step is to prepare the combination dataframe above into the appropriate format to be passed to the runEnergyPlus function().

In [ ]:
inputs = combinations.to_dict("records")

inputs = [(ep_dir, baseline_idf_path, weather_file_path, inputs[i], i) for i in range(n_simulations)]

Run all the simulations with multiprocessing and collect the results.

In [ ]:
# Run all of the simulations in parallel
# Set up multiprocessing by first obtaining the number of processors on your machine.
n_processors = os.cpu_count()

print (f"Preparing to run {n_simulations} EnergyPlus simulation in parallel using {n_processors} processors.")

# Set-up the multiprocessing code block with timers
t0 = time.time()
if __name__ == "__main__":
    with Pool(processes = n_processors) as pool:
        returnValues = pool.starmap(run_energyPlus, inputs)
t1 = time.time()

print (f"\nFinished running all {n_simulations} simulations")

# Un pack the results from the batch simulation
returnCodes = [i[0] for i in returnValues]
hourlyResults = [i[1] for i in returnValues]
resilienceResults = [i[2] for i in returnValues]


# Check if any simulations had errors
errors = [x.args for x in returnCodes if x.returncode == 1]
if len(errors) > 0:
    print (f"The following {len(errors)} simulations had errors:")
    for error in errors:
        print (f"\t{error}")
else:
    print (f"All simulations completed successfully in {t1 - t0:.4f} s.")


### 1.4.3 Save the results

In [ ]:
# Putting both the results dictionaries into a dataframe and concatenating them together
df = pd.DataFrame(hourlyResults)
df2 = pd.DataFrame(resilienceResults)

df = pd.concat([df, df2], axis = 1)

# Save the results file as a csv
savePath = Path("outputs", "results", f"results_{saveName}.csv")
df.to_csv(savePath)

print (f"results dataframe saved to {savePath}.")

### 1.4.4 Visualising Results

I am going to choose to focus on two variables: heatingMax and SET Hours > 30&deg;C for visualization. <span style = "color:lime;"> It is your decision which of the metrics to choose to analyze for the whole set of parameters. </span>

Graphing heatingMax and SET Hours > 30&deg;C for visualization.

In [ ]:
fig, ax = plt.subplots(ncols = 2, width_ratios = [1, 1])

ax[0].plot(combinations[parameterName], df["heatingMax"] / 1000, color = "orange", marker = "o")
ax[1].plot(combinations[parameterName], df["SET > 30°C Degree-Hours [°C·hr]"], color = "red", marker = "o")

ax[0].set_xlabel(parameterName)
ax[0].set_ylabel("Peak Heating Load [kW]")

ax[1].set_xlabel(parameterName)
ax[1].set_ylabel("SET > 30°C Degree-Hours [°C·hr]")

fig.set_figwidth(7.5)
fig.set_figheight(3.5)
fig.tight_layout()

plt.show()

<span style = "color:lime;"> Questions: </span>
* Do the results for this parameter make sense? Are they explainable?
* Are the results significant? How would you judge if they are more significant than other parameters?

## 2. Weighted Sum Analysis
This step shows an example of how to set up a weighted sum analysis with EnergyPlus results. In this step I will use heatingMax and SET Hours > 30&deg;C as in the previous step.

### 2.1 Re-load the results file
Demonstrating how to re-load the results file.

In [ ]:
resultsPath = Path("outputs", "results", f"results_{saveName}.csv")

df = pd.read_csv(resultsPath, index_col = 0)

print (df[["heatingMax", "SET > 30°C Degree-Hours [°C·hr]"]])

For the weighted sum method we have an equation in the form:

$$
\
g(x) = w_1f_1 + w_2f_2 + ... + w_nf_n
\
$$

Where, *f<sub>n</sub>* is the value of the objective function and *w<sub>n</sub>* are the weights given to each function.

Here, I will assume that *w<sub>1</sub>* = 0.9 (heatingMax) and *w<sub>2</sub>* = 0.1 (SET Hours > 30&deg;C). Remember that the weights are to be determined by you.

Calculating for g(x):


In [ ]:
w_1 = 0.9 #heatingMax
w_2 = 0.1 #SET Hours > 30&deg;C

f_1 = df["heatingMax"]
f_2 = df["SET > 30°C Degree-Hours [°C·hr]"]

g = w_1 * f_1 + w_2 * f_2

print (g)

### 2.2 Conversion to a function
It would be handy to convert this into a reusable function as you will need this throughout the course. We will make a function called *costFunction* which takes in two pairs of weights and objective function values and returns the weighted sum.

<span style = "color:dodgerblue;"> Note that I have *w<sub>1</sub>* and *w<sub>2</sub>* as default arguments in the function. This allows you to the option to hard code the chosen weights into the function or pass weights every time you call the function.</span>

In [ ]:
def costFunction(f_1, f_2, w_1 = 0.9, w_2 = 0.1):
    """
    Apply the weighted sum method g(x) = w_1f_1 + w_2f_2
    """
    g = w_1 * f_1 + w_2 * f_2

    return g

Execute the function with default weights

In [ ]:
g = costFunction(f_1, f_2)
print (g)

Execute the function with different weights and also with different EnergyPlus metrics

In [ ]:
print ("With different weights")
g = costFunction(f_1, f_2, w_1 = 0.5, w_2 = 0.5)
print (g)

print ("\nWith different metrics")
g = costFunction(df["heatingSum"], df["HeatIndex:Danger [hr]"], w_1 = 0.5, w_2 = 0.5)
print (g)

## 3. Simulations for the Remaining Parameters

For the coursework, you are asked to peform sensitivity analysis on the remaining parameters. You can perform use *Template_7C.ipynb* as a basis. You have all the basic code you need to generate and run the simulations, collect the results and analyze the results.

Some things to consider:
* You can write separate files or blocks of code to run each parameter. Or you may consider writing a for loop to reduce the amount of duplicated code.
* Decide which metrics you are going to use as part of your multi-objective analysis. Focus on collecting those results.
* Which climate should you do this for?

## 4. Tutorial 7 Summary

In this tutorial, you will have learned:
* How to generate full-factorial, fractional-factorial, and latin hypercube samples for batch EnergyPlus simulations.
* How to produce parametric analysis for EnergyPlus parameters.